NGBoost for outputting Probalitiy Distribution instead of single value

https://stanfordmlgroup.github.io/ngboost/1-useage.html

In [2]:
#Necessary imports
import os
import sys
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ngboost import NGBRegressor
from ngboost.scores import LogScore
from ngboost.distns import  Normal
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import root_mean_squared_error
import optuna
from optuna.trial import TrialState
import uncertainty_toolbox as uct

# check the computer name and set the path accordingly
if os.environ['COMPUTERNAME'] == 'FYNN':            # name of surface PC
    sys.path.append(r'C:\Users\Surface\Masterarbeit')
elif os.environ['COMPUTERNAME'] == 'FYNNS-PC':  # desktop name
    sys.path.append(r'C:\Users\test\Masterarbeit')
    
else:
    raise ValueError("Unbekannter Computername: " + os.environ['COMPUTERNAME'])

from utils.data_prep import load_tranform_and_split_data, set_seed
from utils.metrices import evaluate_intervals

Data Loading and Preprocessing to compare the algorithms, although Decision Trees are not sensitve to feature scalling, they do not use geometric distances


In [3]:
# load and transform the data, split it into training, validation, and test sets
# uses random state 42 internally for reproducibility
# the split ratio is 60% training, 20% validation, and 20%
# return the feature names for later use

X_train, X_val, X_test, y_train, y_val, y_test, feature_names = load_tranform_and_split_data('C1_V01_delta_kan', 
                                                                                                       split_ratio=(0.6, 0.2, 0.2))

Optuna Hyperparameter Search

In [ ]:
def objective(trial):

    # Define the hyperparameters to tune
    n_estimators = trial.suggest_int("n_estimators", 50, 1000, step = 50)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2,log=True)
    max_depth = trial.suggest_int("max_depth", 4, 20)
    
    # Create the NGBRegressor model with the suggested hyperparameters
    ngb = NGBRegressor(
        Dist=Normal, 
        Score=LogScore, 
        Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=max_depth),
        verbose=False, 
        n_estimators=n_estimators, 
        learning_rate=learning_rate
    )
    
    # Fit the model to the training data
    ngb.fit(X_train, y_train, X_val=X_val, Y_val=y_val, early_stopping_rounds=20)
    
    # Make predictions on the validation set
    y_preds = ngb.predict(X_val)
    y_dists = ngb.pred_dist(X_val)
    
    # Calculate the negative log likelihood
    nll = -y_dists.logpdf(y_val).mean()
    rmse = root_mean_squared_error(y_val, y_preds)
    print(f"Trial {trial.number}: NLL={nll}, RMSE={rmse}")
    
    return nll

Execute Study

In [ ]:
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(),  # Use TPE sampler for hyperparameter optimization
    pruner=optuna.pruners.MedianPruner(        
        n_startup_trials=20,                                    # Number of trials to run before pruning starts
        n_warmup_steps=5                                        # Number of warmup steps before pruning starts
    )
)
study.optimize(objective, n_trials=200, timeout=None, n_jobs=1, show_progress_bar=True)

print(study.best_params)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Train final Model

In [ ]:
SEED = 42
set_seed(SEED)
ngb = NGBRegressor(Dist=Normal, 
                   Score = LogScore, 
                   Base = DecisionTreeRegressor(criterion='friedman_mse',                                                
                                                 max_depth=9,
                                                 random_state=SEED), 
                   verbose = True, 
                   n_estimators=750, 
                   learning_rate=0.003,
                   random_state=SEED)

ngb.fit(X_train, y_train, X_val=X_val, Y_val=y_val, early_stopping_rounds=50)

Evalute 1 Run Using Uncertainty Toolbox to get metrices

In [ ]:
set_seed(SEED)
y_test_preds = ngb.predict(X_test)
y_test_dists = ngb.pred_dist(X_test)
# Extract standard deviation from test predictions
test_stddev = y_test_dists.scale

# Calculate and print all metrics inclunding RMSE, MAE, R²-Score, NLL, CRPS
pnn_metrics = uct.metrics.get_all_metrics( y_test_preds, test_stddev,y_test)
print(pnn_metrics)
#print(pnn_metrics['accuracy']['rmse'])
# Calculate coverage for 95% confidence interval
coverage_95 = uct.metrics_calibration.get_proportion_in_interval(y_test_preds, test_stddev, y_test, quantile = 0.95 )
print(f"Coverage 95%: {coverage_95}")

# use own function to calculate coverage and MPIW
ev_intervals = evaluate_intervals(y_test_preds, test_stddev, y_test, coverage=0.95)
print(f'coverage: {ev_intervals["coverage"]}, MPIW: {ev_intervals["MPIW"]}')

# calibration Curve with UCT
uct.viz.plot_calibration(y_test_preds, test_stddev, y_test)

# adversarial group calibration
uct.viz.plot_adversarial_group_calibration(y_test_preds, test_stddev, y_test)

Do 10 Runs for a more representative Study

In [8]:
results_list = []
predictions_list = []
list_of_seeds = [42, 123, 777, 2024, 5250, 8888, 9876, 10001, 31415, 54321]
ngb_prediction_path = r"C:\Users\test\Masterarbeit\models\Modelresults\NGBoost"
ngb_result_path = r"C:\Users\test\OneDrive\Master Management und Engineering\Masterarbeit\Experimente\Evaluation\10 Runs\NGBoost"

for run, seed in enumerate(list_of_seeds):

    print(f"Run {run+1} with seed {seed}")
    set_seed(seed)

    # Define model, use seed to intialize the base learner and ngboost
    ngb = NGBRegressor(Dist=Normal, 
                        Score = LogScore, 
                        Base = DecisionTreeRegressor(criterion='friedman_mse',
                                                        max_depth=9,
                                                        random_state=seed), 
                        verbose = True, 
                        n_estimators=750, 
                        learning_rate=0.003,
                        random_state=seed)
    
    # Train and evaluate the model
    ngb.fit(X_train, y_train, X_val=X_val, Y_val=y_val, early_stopping_rounds=50)

    # Evaluate on test set
    y_test_preds = ngb.predict(X_test)
    y_test_dists = ngb.pred_dist(X_test)
    # Extract standard deviation from test predictions
    test_stddev = y_test_dists.scale

    # Calculate and print all metrics inclunding RMSE, MAE, R²-Score, NLL, CRPS
    pnn_metrics = uct.metrics.get_all_metrics( y_test_preds, test_stddev,y_test)
    print(pnn_metrics)

    # use own function to calculate coverage and MPIW
    ev_intervals = evaluate_intervals(y_test_preds, test_stddev, y_test, coverage=0.95)
    print(f'coverage: {ev_intervals["coverage"]}, MPIW: {ev_intervals["MPIW"]}')

    predictions_per_run = {
        'mean_prediction': y_test_preds,
        'std_prediction': test_stddev,
    }

    results_per_run = {
    'RMSE': pnn_metrics['accuracy']['rmse'],
    'MAE': pnn_metrics['accuracy']['mae'],
    'R2': pnn_metrics['accuracy']['r2'], 
    'Correlation' : pnn_metrics['accuracy']['corr'],
    'NLL': pnn_metrics['scoring_rule']['nll'],
    'CRPS': pnn_metrics['scoring_rule']['crps'],
    'coverage': ev_intervals["coverage"],
    'MPIW': ev_intervals["MPIW"],
    }

    predictions_list.append(predictions_per_run)
    results_list.append(results_per_run)
#save the predictions 
with open(os.path.join(ngb_prediction_path, "ngboost_predictions_list.pkl"), "wb") as f:
    pickle.dump(predictions_list, f)

#save the results in an excel file
results_df = pd.DataFrame(results_list)
results_df.to_excel(os.path.join(ngb_result_path, "ngboost_results.xlsx"), index=False)

Run 1 with seed 42
[iter 0] loss=3.4547 val_loss=3.4396 scale=1.0000 norm=6.1507


KeyboardInterrupt: 

In [4]:
ngb_prediction_path = r"C:\Users\test\Masterarbeit\models\Modelresults\NGBoost"
ngb_result_path = r"C:\Users\test\OneDrive\Master Management und Engineering\Masterarbeit\Experimente\Evaluation\10 Runs\NGBoost"

with open(os.path.join(ngb_prediction_path, "ngboost_predictions_list.pkl"), "rb") as f:
    predictions_list = pickle.load(f)

mean_list = []
std_list = []

for id, run in enumerate(predictions_list):
    # extract mean and std predictions
    mean = run['mean_prediction']
    std = run['std_prediction']
    
    # append to lists
    mean_list.append(mean)
    std_list.append(std)
    
    # calibration Curve with UCT
    uct.viz.plot_calibration(mean, std, y_test)
    plt.savefig(os.path.join(ngb_result_path, f"calibration_run_{id+1}.svg"), format ='svg')
    plt.savefig(os.path.join(ngb_result_path, f"calibration_run_{id+1}.png"), format ='png')
    plt.close()

    # adversarial group calibration
    uct.viz.plot_adversarial_group_calibration(mean, std, y_test)
    plt.savefig(os.path.join(ngb_result_path, f"adversarial_group_calibration_run_{id+1}.svg"), format ='svg')
    plt.savefig(os.path.join(ngb_result_path, f"adversarial_group_calibration_run_{id+1}.png"), format ='png')
    plt.close()

# predictions_list enthält pro Run ein Array mit 10403 Werten
mean_matrix = np.array(mean_list)  # Shape: (n_runs, 10403)
std_matrix = np.array(std_list)    # Shape: (n_runs, 10403)

# Mittelwert und Std für jeden Datenpunkt über alle Runs
mean_per_datapoint = np.mean(mean_matrix, axis=0)  # Shape: (10403,)
std_per_datapoint = np.mean(std_matrix, axis=0)    # Shape: (10403,)

# calibration Curve with UCT
uct.viz.plot_calibration(mean_per_datapoint, std_per_datapoint, y_test)
plt.savefig(os.path.join(ngb_result_path, "calibration_run_mean.svg"), format ='svg')
plt.savefig(os.path.join(ngb_result_path, "calibration_run_mean.png"), format ='png')
plt.close()

# adversarial group calibration
uct.viz.plot_adversarial_group_calibration(mean_per_datapoint, std_per_datapoint, y_test)
plt.savefig(os.path.join(ngb_result_path, "adversarial_group_calibration_run_mean.svg"), format ='svg')
plt.savefig(os.path.join(ngb_result_path, "adversarial_group_calibration_run_mean.png"), format ='png')
plt.close()
